In [1]:
# NEED TO MAKE SURE WE ARE IN CORRECT DIRECTORY FOR NOTEBOOK
import os
if os.getcwd() == 'c:\\Users\\jafar\\Desktop\\V2022\\Master2022\\UserSimCRS\\usersimcrs':
  os.chdir("..") # Change to root level directory
os.getcwd()


'c:\\Users\\jafar\\Desktop\\V2022\\Master2022\\UserSimCRS'

In [2]:
# IMPORTS
"""Console application for running simulation."""

import argparse
import configparser
import os
import sys
import json
import yaml
import datetime

from dialoguekit.nlu.models.diet_classifier_rasa import IntentClassifierRasa
from dialoguekit.nlu.models.intent_classifier_cosine import IntentClassifierCosine
from dialoguekit.nlg.nlg import NLG
from dialoguekit.nlu.nlu import NLU
from dialoguekit.agent.agent import Agent
from dialoguekit.core.dialogue import Dialogue
from dialoguekit.core.ontology import Ontology
from dialoguekit.core.recsys.item_collection import ItemCollection
from dialoguekit.core.recsys.ratings import Ratings
from dialoguekit.manager.dialogue_manager import DialogueManager
from dialoguekit.platforms.platform import Platform
from dialoguekit.agent.terminal_agent import TerminalAgent
from dialoguekit.agent.moviebot_agent import MovieBotAgent
from dialoguekit.core.intent import Intent
from dialoguekit.core.utterance import Utterance
from dialoguekit.nlu.models.satisfaction_classifier import SatisfactionClassifier

from usersimcrs.simulator.user_simulator import UserSimulator
from usersimcrs.sample_agent.sample_agent import SampleAgent
from usersimcrs.simulator.agenda_based_simulator import AgendaBasedSimulator
from usersimcrs.simulator.preference_model import (
    PreferenceModel,
    PreferenceModelVariant,
)
from usersimcrs.simulator.interaction_model import InteractionModel
from usersimcrs.utils.persona_generator import Persona


In [3]:
# FUNCTIONS
def simulate_conversation(
    agent: Agent, user_simulator: UserSimulator
) -> Dialogue:
    """Simulates a single conversation and returns the resulting dialogue.

    Args:
        agent: An agent.
        user_simulator: A user simulator.
    """
    platform = Platform()  # TODO: Add simulator platform
    dm = DialogueManager(agent, user_simulator, platform)
    agent.connect_dialogue_manager(dialogue_manager=dm)
    user_simulator.connect_dialogue_manager(dialogue_manager=dm)
    dm.start()
    dm.close()
    return dm.dialogue_history

In [4]:
# READ CONFIG
conf_parser = argparse.ArgumentParser(
    description=__doc__, # printed with -h/--help
    # Don't mess with format of description
    formatter_class=argparse.RawDescriptionHelpFormatter,
    # Turn off help, so we print all options in response to -h
    add_help=False
    )
conf_file = "configs\config.ini"
cp = configparser.ConfigParser()
cp.read([conf_file])
input_file_paths = dict(cp.items('SETTINGS'))

In [5]:
# INITIALISE OBJECTS
# agent = SampleAgent(agent_id="sample_agent")
# agent = TerminalAgent("test")

ontology = Ontology(input_file_paths["ontology"])

item_collection = ItemCollection()
item_collection.load_items_csv(input_file_paths["items"],["ID", "NAME", "genres","keywords"])

ratings = Ratings(item_collection=item_collection)
ratings.load_ratings_csv(input_file_paths["ratings"])

with open(input_file_paths["dialogues"]) as annotated_dialogues_file:
  annotated_conversations = json.load(annotated_dialogues_file)
gt_intents = []
utterances = []
for conversation in annotated_conversations:
    for turn in conversation["conversation"]:
        if turn["participant"] == "AGENT":
            gt_intents.append(Intent(turn["intent"]))
            utterances.append(Utterance(turn["utterance"]))
with open(input_file_paths["ir"]) as yaml_file:
  config = yaml.load(yaml_file, Loader = yaml.FullLoader)
agent_intents = [Intent(x) for x in config["agent_intents"]]

preference_model = PreferenceModel(
  ontology,
  item_collection,
  ratings,
  PreferenceModelVariant.PKG,
  historical_user_id="17",
)

interaction_model = InteractionModel(
  input_file_paths["ir"], annotated_conversations
)

satisfaction_model = SatisfactionClassifier()

nluc = IntentClassifierCosine(agent_intents)
nluc.train_model(utterances,gt_intents)

persona = Persona("Jafar","17",False,(datetime.time(18,0,0),datetime.time(21,0,0)),True,3,0.6,2)

c:\Users\jafar\anaconda3\envs\usersimcrs\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LinearSVC from version 0.24.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\Users\jafar\anaconda3\envs\usersimcrs\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
# TRAIN OBJECTS THAT REQUIRE IT
nlur = IntentClassifierRasa(
  agent_intents,
  "data/agents/moviebot/annotated_dialogues_v2_rasa_agent.yaml",
  ".rasa",
)
nlur.train_model()

nlu = NLU(nluc,[nlur])
nlg = NLG()
nlg.template_from_file(template_file=input_file_paths["dialogues"], participant_to_learn="USER", satisfaction_classifier= satisfaction_model)

WhitespaceTokenizer_0
CountVectorsFeaturizer_1


c:\Users\jafar\anaconda3\envs\usersimcrs\lib\site-packages\rasa\utils\train_utils.py:530: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss.
  category=UserWarning,
c:\Users\jafar\anaconda3\envs\usersimcrs\lib\site-packages\rasa\utils\tensorflow\model_data_utils.py:396: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array([v[0] for v in values]), number_of_dimensions=3
c:\Users\jafar\anaconda3\envs\usersimcrs\lib\site-packages\rasa\utils\tensorflow\model_data.py:740: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=o

In [7]:
# INTIALISE SIMULATOR
agent = MovieBotAgent("14","http://152.94.232.109:5001")
simulator = AgendaBasedSimulator(
    "simulator",
    preference_model,
    interaction_model,
    nlu,
    nlg,
    ontology,
    item_collection,
    ratings,
    persona,
    satisfaction_model
)

AGENDA:  [Intent(COMPLETE), Intent(NOTE.NOTE-ACCEPT), Intent(INQUIRE.MORE), Intent(INQUIRE.MORE), Intent(INQUIRE.ITEMINFO), Intent(DISCLOSE), Intent(DISCLOSE), Intent(DISCLOSE), Intent(INQUIRE), Intent(DISCLOSE), Intent(DISCLOSE), Intent(DISCLOSE), Intent(REVEAL.REVISE), Intent(DISCLOSE), Intent(DISCLOSE), Intent(DISCLOSE.NON-DISCLOSE)]


In [8]:
# RUN SIMULATION
simulate_conversation(agent,simulator)

AGENT: Hi there. I am IAI MovieBot, your movie recommending buddy. I can recommend you movies based on your preferences.
 I will ask you a few questions and based on your answers, I will try to find a movie for you.


88: agent intent DISCLOSE.NON-DISCLOSE
89: agent annotations []
97: satisfaction score 2
99: expected responses: DISCLOSE.NON-DISCLOSE ['INQUIRE', 'INQUIRE.ELICIT', 'DISCLOSE.NON-DISCLOSE']
Agenda expected
AGENDA UPDATE: DISCLOSE.NON-DISCLOSE
112:response intent DISCLOSE.NON-DISCLOSE
151: response Hello.
USER:  Hello.

ELICIT
AGENT: Do you have any specific genres in mind?
88: agent intent INQUIRE.ELICIT
89: agent annotations [SlotValueAnnotation(REQUEST, genres,             25, 31)]
97: satisfaction score 2
99: expected responses: DISCLOSE.NON-DISCLOSE ['INQUIRE', 'INQUIRE.ELICIT', 'DISCLOSE.NON-DISCLOSE']
Agenda expected
AGENDA UPDATE: DISCLOSE
112:response intent DISCLOSE
122: user slot values [Annotation(GENRE, Crime), Annotation(GENRE, Drama), Annotation(GENRE, War)]

AttributeError: 'bool' object has no attribute 'text'

In [ ]:
interaction_model.initialize_agenda()

[Intent(NOTE),
 Intent(DISCLOSE),
 Intent(DISCLOSE),
 Intent(NOTE),
 Intent(INQUIRE.SIMILAR),
 Intent(NOTE),
 Intent(NOTE),
 Intent(DISCLOSE),
 Intent(DISCLOSE.NON-DISCLOSE)]

In [10]:
preference_model._item_preferences.get_preferences("ITEM_ID")

{'6': 0.5555555555555556,
 '110': 0.5555555555555556,
 '377': 0.11111111111111116,
 '457': 1.0,
 '480': 0.11111111111111116,
 '648': 0.11111111111111116,
 SlotValueAnnotation(TITLE, Heat,             13, 17): -1}